In [ ]:
%load_ext blackcellmagic

In [ ]:
import topo as tg
import importlib
import uuid
importlib.reload(tg)
node_list = [
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {}),
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {}),
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {}),
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {}),
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {}),
    tg.Node(str(uuid.uuid4()), "host", 1000, 1, int(1e9), 0, 0, {})
]
t = tg.Topology()
for n in node_list:
    t.add_node(n)
t.connect(node_list[0], node_list[1], uuid.uuid4(), int(1e7), 10)
t.connect(node_list[1], node_list[2], uuid.uuid4(), int(1e7), 2)
t.connect(node_list[1], node_list[3], uuid.uuid4(), int(1e7), 2)
t.connect(node_list[1], node_list[4], uuid.uuid4(), int(1e7), 2)
t.connect(node_list[4], node_list[5], uuid.uuid4(), int(1e7), 1)

In [ ]:
import vivaldi.compute as vivaldi_compute
import vivaldi.coordinate as vivaldi_coordinate
importlib.reload(vivaldi_compute)
importlib.reload(vivaldi_coordinate)
lat_matrix = vivaldi_compute.get_latency_matrix(t, [n.uuid for n in t.get_nodes()])
print(lat_matrix)
coordinate_2d_class = vivaldi_coordinate.create_coordinate_class(2)
matrix_error = vivaldi_compute.matrix_error(lat_matrix, {n.uuid: coordinate_2d_class(0,0) for n in node_list})
print(matrix_error)

In [ ]:
updated = vivaldi_compute.vivaldi_compute(t, {n.uuid: coordinate_2d_class(0,0) for n in node_list}, 0.01, 100)
for k, v in updated.items():
    print(k, v)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect(1)
plt.scatter(*zip(*updated.values()))
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
coords = list(updated.values())
matrix = np.zeros((len(coords), len(coords)))
for i in range(len(coords)):
    for j in range(len(coords)):
        matrix[i][j] = abs(coords[i] - coords[j])
print(matrix)

In [ ]:
import yaml
import importlib
import topo
importlib.reload(topo)
data = yaml.load(open('samples/a0.yaml', 'r').read(), Loader=yaml.Loader)
sc = topo.Scenario.from_dict(data)

In [ ]:
import networkx as nx
nx.draw(sc.topo.g)

In [ ]:
coordinate_3d_class = vivaldi_coordinate.create_coordinate_class(3)
coordinate_4d_class = vivaldi_coordinate.create_coordinate_class(4)
init_coords = {n.uuid: coordinate_3d_class(0,0,0) for n in sc.topo.get_hosts()}
coords = vivaldi_compute.vivaldi_compute(sc.topo, init_coords, 1, 500)

In [ ]:
from functools import partial

cloud_node = sc.topo.get_node(list(
    filter(
        partial(sc.topo.label_filter, {"machine": "rack"}),
        [h.uuid for h in sc.topo.get_hosts()],
    )
)[0])
base_coord = coords[cloud_node.uuid]
coords = {k: v - base_coord for k, v in coords.items()}
for k, v in coords.items():
    print(k, v)

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.set_aspect(1)
ax.scatter3D(*zip(*coords.values()))
# plt.grid(True)
plt.show()